In [1]:
# Run if working locally
%load_ext autoreload
%autoreload 2
%load_ext nb_black

<IPython.core.display.Javascript object>

In [97]:
import sqlite3
from sqlite3 import Error
import pickle
import os, sys
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, config.root_path)

from src.dataset.dataset import RawData
from src.dataset.wikisection_preprocessing import (
    tokenize,
    clean_sentence,
    preprocess_text_segmentation,
    format_data_for_db_insertion,
)
from src.dataset.utils import truncate_by_token
from db.dbv2 import Table, AugmentedTable, TrainTestTable
import pprint

from utils.metrics import windowdiff, pk

from src.bertkeywords.src.similarities import Embedding, Similarities
from src.bertkeywords.src.keywords import Keywords
from src.encoders.coherence import Coherence
from src.dataset.utils import flatten, dedupe_list, truncate_string
from src.experimentation.coherence import SimpleExperiment, CoherenceExperiment

import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [102]:
experimentation = SimpleExperiment()

experimentation.queue_experiment(
    CoherenceExperiment(
        model_string="bert-base-uncased",
        max_words_per_step=3,
        start=10000,
        num_samples=5000,
        same_word_multiplier=2,
        no_same_word_penalty=2,
    )
)
experimentation.queue_experiment(
    CoherenceExperiment(
        model_string="bert-base-uncased",
        max_words_per_step=4,
        start=10000,
        num_samples=5000,
        same_word_multiplier=2,
        no_same_word_penalty=2,
    )
)

<IPython.core.display.Javascript object>

In [ ]:
experimentation.run()

Running experiment set: I8TaE
Running experiment: CoherenceExperiment(num_samples=5000, start=10000, dataset_type='city', model_string='bert-base-uncased', max_words_per_step=3, same_word_multiplier=2, no_same_word_penalty=2, prediction_threshold=0.25, coherence_dump_on_prediction=False, pruning=1, pruning_min=6, dynamic_threshold=False, threshold_warmup=10, last_n_threshold=5, kb_embeddings=False, experiment_hash='OXLkP')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /Users/amitmaraj/.cache/torch/sentence_transformers/bert-base-uncased were not used when initializing BertModel: ['

.....................

In [60]:
import pandas as pd
from dataclasses import asdict, dataclass

df = pd.json_normalize(asdict(obj) for obj in experiment.get_experiments())

<IPython.core.display.Javascript object>

In [61]:
df

,num_samples,start,dataset_type,model_string,max_words_per_step,same_word_multiplier,no_same_word_penalty,prediction_threshold,coherence_dump_on_prediction,pruning,pruning_min,dynamic_threshold,threshold_warmup,last_n_threshold,kb_embeddings,experiment_hash
0,10,0,city,bert,4,2,2,0.25,False,1,6,False,10,5,False,None
1,10,0,city,bert,4,2,2,0.25,False,1,6,False,10,5,False,None


<IPython.core.display.Javascript object>

In [62]:
df.append(pd.json_normalize(asdict(obj) for obj in experiment.get_experiments()))

,num_samples,start,dataset_type,model_string,max_words_per_step,same_word_multiplier,no_same_word_penalty,prediction_threshold,coherence_dump_on_prediction,pruning,pruning_min,dynamic_threshold,threshold_warmup,last_n_threshold,kb_embeddings,experiment_hash
0,10,0,city,bert,4,2,2,0.25,False,1,6,False,10,5,False,None
1,10,0,city,bert,4,2,2,0.25,False,1,6,False,10,5,False,None
0,10,0,city,bert,4,2,2,0.25,False,1,6,False,10,5,False,None
1,10,0,city,bert,4,2,2,0.25,False,1,6,False,10,5,False,None


<IPython.core.display.Javascript object>

In [55]:
results = asdict(experiment.get_experiments()[0])

<IPython.core.display.Javascript object>

In [56]:
results["predictions"] = []

<IPython.core.display.Javascript object>

In [59]:
pd.json_normalize(results)

,num_samples,start,dataset_type,model_string,max_words_per_step,same_word_multiplier,no_same_word_penalty,prediction_threshold,coherence_dump_on_prediction,pruning,pruning_min,dynamic_threshold,threshold_warmup,last_n_threshold,kb_embeddings,predictions
0,10,0,city,bert,4,2,2,0.25,False,1,6,False,10,5,False,[]


<IPython.core.display.Javascript object>